#  Import Libraries

In [ ]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import *
from keras import backend as K

Using TensorFlow backend.


# Load MNIST Dataset

In [ ]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

print("MNIST training set  len ={}".format(len(X_train)))
print("MNIST training set  len ={}".format(len(X_test)))

11493376/11490434 [==============================] - 1s 0us/step
MNIST training set  len =60000
MNIST training set  len =10000


# Scalling Images

In [ ]:
#--------------------------------------
# Scalling Images by 255
#--------------------------------------
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

#  Change Output to Categorical

In [ ]:
#--------------------------------------
# Change Output to Categorical
#--------------------------------------
num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
y_train[:5, :]

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]], dtype=float32)

#  MNIST-MLP

##  Build MLP layers

In [ ]:
#--------------------------------------
# Build MLP NN mlp
#--------------------------------------
# define the mlp
mlp = Sequential()
mlp.add(Flatten(input_shape=X_train.shape[1:]))
mlp.add(Dense(512, activation='relu'))
mlp.add(Dropout(0.2))
mlp.add(Dense(512, activation='relu'))
mlp.add(Dropout(0.2))
mlp.add(Dense(10, activation='softmax'))

mlp.compile(loss='categorical_crossentropy', optimizer='adam', 
    metrics=['accuracy'])

# summarize the mlp
mlp.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5130      
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0
________________________________________________

##  Trainning MLP

In [ ]:
batch_size = 128
epochs = 12
mlp.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 3s 57us/step - loss: 0.2506 - accuracy: 0.9250 - val_loss: 0.1080 - val_accuracy: 0.9665
Epoch 2/12
60000/60000 [==============================] - 2s 29us/step - loss: 0.1020 - accuracy: 0.9679 - val_loss: 0.0763 - val_accuracy: 0.9765
Epoch 3/12
60000/60000 [==============================] - 2s 28us/step - loss: 0.0719 - accuracy: 0.9769 - val_loss: 0.0824 - val_accuracy: 0.9758
Epoch 4/12
60000/60000 [==============================] - 2s 28us/step - loss: 0.0555 - accuracy: 0.9823 - val_loss: 0.0690 - val_accuracy: 0.9785
Epoch 5/12
60000/60000 [==============================] - 2s 28us/step - loss: 0.0461 - accuracy: 0.9853 - val_loss: 0.0814 - val_accuracy: 0.9765
Epoch 6/12
60000/60000 [==============================] - 2s 29us/step - loss: 0.0392 - accuracy: 0.9869 - val_loss: 0.0762 - val_accuracy: 0.9783
Epoch 7/12
60000/60000 [==============================] - 2s 28us/st

## Model Evaluation

In [ ]:
score = mlp.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.07689996974503974
Test accuracy: 0.9800999760627747


## Model Saving 

In [ ]:
mlp.save('MLP-MNIST.h5')

#  MNIST - CNN

##  Prepare Image for CNN

In [ ]:
K.image_data_format()

'channels_last'

In [ ]:
# input image dimensions
img_rows, img_cols = 28, 28
if K.image_data_format() == 'channels_first':
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

 ## Build CNN

In [ ]:
cnn = Sequential()
cnn.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
cnn.add(Conv2D(64, (3, 3), activation='relu'))
cnn.add(MaxPooling2D(pool_size=(2, 2)))
cnn.add(Dropout(0.25))
cnn.add(Flatten())
cnn.add(Dense(128, activation='relu'))
cnn.add(Dropout(0.5))
cnn.add(Dense(num_classes, activation='softmax'))

cnn.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
cnn.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)              

## Training CNN

In [ ]:
cnn.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 8s 132us/step - loss: 0.2648 - accuracy: 0.9182 - val_loss: 0.0560 - val_accuracy: 0.9827
Epoch 2/12
60000/60000 [==============================] - 3s 56us/step - loss: 0.0904 - accuracy: 0.9732 - val_loss: 0.0415 - val_accuracy: 0.9864
Epoch 3/12
60000/60000 [==============================] - 3s 56us/step - loss: 0.0671 - accuracy: 0.9799 - val_loss: 0.0366 - val_accuracy: 0.9877
Epoch 4/12
60000/60000 [==============================] - 3s 56us/step - loss: 0.0567 - accuracy: 0.9834 - val_loss: 0.0310 - val_accuracy: 0.9892
Epoch 5/12
60000/60000 [==============================] - 3s 56us/step - loss: 0.0480 - accuracy: 0.9855 - val_loss: 0.0306 - val_accuracy: 0.9906
Epoch 6/12
60000/60000 [==============================] - 3s 56us/step - loss: 0.0421 - accuracy: 0.9869 - val_loss: 0.0279 - val_accuracy: 0.9905
Epoch 7/12
60000/60000 [==============================] - 3s 58us/s

## Model Evaluation 

In [ ]:
score = cnn.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.031298850555110495
Test accuracy: 0.9908999800682068


##  Model Saving

In [ ]:
cnn.save('CNN-MNIST.h5')